<a href="https://colab.research.google.com/github/withtaylors/Samsung_Codes/blob/main/Samsung_llm_%EC%A7%80%EC%97%B0_%EC%8B%9C%EB%8F%843.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# GPU 로 설정 바꾸기
# pip install 돌리고 런타임 다시 시작하기!
!pip install peft
!pip install Faiss-cpu
!pip install langchain
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

!pip install rank_bm25
!pip install sentencepiece
!pip install sentence-transformers
!pip install pypdf
!pip install "unstructured[all-docs]"
!pip install elasticsearch openai tiktoken langchain
!pip install elasticsearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ..

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
from peft import PeftModel, PeftConfig
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.llms import OpenAIChat
from langchain.chat_models import ChatOpenAI

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import TextLoader

import torch
import re
import base64
import os
import pandas as pd

In [ ]:
# SEARCH_NUM = 3 : 검색 결과나 처리할 항목의 수를 정의
# conversation_pairs = []: 빈 리스트, 대화나 상호작용과 관련된 항목 쌍을 저장
# loaded_memory = "": 빈 문자열, 대화 기록이나 유사한 데이터를 저장
# dialogues = "": 빈 문자열, 대화 관련 내용이나 텍스트 데이터를 저장
# query2 = "": 빈 문자열, 검색이나 검색 맥락에서의 요청이나 질문을 저장

SEARCH_NUM = 3
conversation_pairs = []
loaded_memory =""
dialogues = ""
query2=""

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# textfile load
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma
loader = DirectoryLoader("/content/drive/MyDrive/finaltexts", loader_cls=TextLoader)

data=loader.load()

# 디렉토리 경로 설정
directory_path = '/content/drive/MyDrive/finaltexts'

# 검색할 키워드 목록
keywords = ['자동차', '하이브', '미국', '퀄컴', '롯데칠성']

# 해당 디렉토리 내의 모든 파일 목록을 가져오기
file_list = os.listdir(directory_path)

# 키워드가 포함된 파일 찾기
for file in file_list:
    with open(os.path.join(directory_path, file), 'r', encoding='utf-8') as f:
        content = f.read()
        if any(keyword in content for keyword in keywords):
            print(file)

# 파일 리스트의 갯수 출력
print("파일 수:", len(file_list))

하이브_pdf 텍스트.txt
하이브_ 2023년 발매 앨범 및 콘서트1.txt
하이브_ONE-YEAR PERFORMANCE.txt
하이브_최근 투자의견 및 목표주가 변경 (수정주가 기준).txt
하이브_KEY CHANGES.txt
하이브_포괄손익계산서.txt
하이브_현금흐름표.txt
하이브_Valuation summary.txt
하이브_재무상태표.txt
하이브 소속 아티스트 3분기 국내 앨범 판매량.txt
하이브_AT A GLANCE.txt
하이브_이익 추정치 변경.txt
하이브_2023년 발매 앨범 및 콘서트2.txt
하이브_실적 추이와 전망.txt
하이브_SOTP 밸류에이션.txt
하이브_재무비율 및 주당지표.txt
하이브_분기 실적.txt
퀄컴_포괄손익계산서.txt
퀄컴_요약 손익 계산서 (Non-GAAP 기준).txt
퀄컴_AT A GLANCE.txt
퀄컴_ONE-YEAR PERFORMANCE.txt
퀄컴_재무비율 및 주당지표.txt
퀄컴_재무상태표.txt
퀄컴_현금흐름표.txt
퀄컴_SUMMARY OF FINANCIAL DATA.txt
퀄컴_부문별 breakdown (Non-GAAP 기준).txt
퀄컴_pdf 텍스트.txt
롯데칠성_ONE YEAR PERFORMANCE.txt
롯데칠성_AT A GLANCE.txt
롯데칠성_분기실적.txt
롯데칠성

In [ ]:
# text split
# 문서를 크기가 100, 중복이 30인 청크로 나눔
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=30)
# 분할기를 사용하여 로드된 데이터를 청크로 나눔
data_split = text_splitter.split_documents(data)

# embedding
# Hugging Face 모델(jhgan/ko-sroberta-multitask)을 사용하여 임베딩을 생성
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
# FAISS를 사용하여 분할된 텍스트에 대한 벡터 저장소를 생성
dbf = FAISS.from_documents(data_split, embeddings)
docs_memory = []
cnt=0

# 특정 조건(예: 특정 단어나 구문이 발견될 때)에 따라 프로세스를 중지하는 사용자 정의 정지 기준을 구현
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(그만):])).item():
                return True

        return False

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
# graphfile load
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma
loader2 = DirectoryLoader("/content/drive/MyDrive/finalgraphs", loader_cls=TextLoader)

data2=loader2.load()

# 디렉토리 경로 설정
directory_path2 = '/content/drive/MyDrive/finalgraphs'

# 검색할 키워드 목록
keywords = ['자동차', '하이브', '미국', '퀄컴', '롯데칠성']

# 해당 디렉토리 내의 모든 파일 목록을 가져오기
file_list2 = os.listdir(directory_path2)

# 키워드가 포함된 파일 찾기
for file2 in file_list2:
    with open(os.path.join(directory_path2, file2), 'r', encoding='utf-8') as f:
        content = f.read()
        if any(keyword in content for keyword in keywords):
            print(file2)

# 파일 리스트의 갯수 출력
print("파일 수:", len(file_list2))

미국 마감시황_국채 10년물 3개월 추이.txt
미국 마감시황_미국 3분기 비농업부문 노동생산성.txt
미국 마감시황_업종별 수익률.txt
미국 마감시황_미국 연속 실업수당 신청 건수.txt
미국 마감시황_달러 인덱스 3개월 추이.txt
미국 마감시황_S_P500 3개월 추이.txt
자동차_현대차 월간 도매 판매.txt
자동차_국내 현대 팰리세이드 _ 싼타페 vs 기아 쏘렌토 판매.txt
자동차_산업 평균 인센티브.txt
자동차_기아차 분기별 도매 판매.txt
자동차_현대차 분기별 도매 판매.txt
자동차_미국 딜러 재고 추이.txt
자동차_미국 시장 전기차 MS 현대차 vs 기아.txt
자동차_냉연강판 가격 추이.txt
자동차_현대차 미국 시장, 전체 MS vs 전기차 MS.txt
자동차_중국 딜러 재고 추이.txt
자동차_기아차 월간 도매 판매.txt
자동차_미국 맨하임 중고차 가격 지수.txt
하이브_그래프_2년간 목표주가 변경 추이.txt
하이브_그래프_위버스 ARPPU 트렌드.txt
하이브_그래프_위버스 MAU 추이.txt
하이브_그래프_이익 추정치 변경.txt
퀄컴_그래프_매출액과 

In [ ]:
# text split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=30)
data_split2 = text_splitter.split_documents(data2)

# embedding
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
dbg = FAISS.from_documents(data_split2, embeddings)
docs_memory = []
cnt=0
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(그만):])).item():
                return True

        return False

In [ ]:
data_split[3].page_content

'하이브는 4분기에 주요 아티스트의 촘촘한 활동이 예정되어 있고 빌리프랩 연결 계상 예정이며 24년에도 다수의 신인 데뷔로 성장성 강화될 전망이다.'

In [ ]:
data_split[0].metadata['source'].split

<function str.split(sep=None, maxsplit=-1)>



---
리트리버


---





In [ ]:
from rank_bm25 import BM25Okapi
def make_tok(sent):
  return sent.split(" ")

# text
corpus=  [i.page_content for i in data_split]
tokenized_corpus = [make_tok(i.page_content) for i in data_split]
bm25 = BM25Okapi(tokenized_corpus)

# graph
corpus2=  [i.page_content for i in data_split2]
tokenized_corpus2 = [make_tok(i.page_content) for i in data_split2]
bm25g = BM25Okapi(tokenized_corpus2)

In [ ]:
# search similarity - text
def db_search(query: str, k: int):
    docs = dbf.similarity_search(query, k)
    return docs

# search similarity - graph
def db_search2(query: str, k: int):
    docs = dbg.similarity_search(query, k)
    return docs



---
query


---




In [ ]:
query='위버스 설명'
docs_bm25=bm25.get_top_n(make_tok(query),corpus,n=2)
docs_bm25

['하이브의 위버스의 가치는 3,460.1십억원이다. \n하이브의 위버스 지분율은 55.45%이다.\n\n하이브의 투자지분의 가치는 2Q23 장부가 기준으로 331.9십억원이다.',
 '하이브의 위버스 서비스 기능 추가로 시장 영향력 확대를 지속했다.\n하이브는 글로벌 엔터 시장 내 영향력 강화 지속할 전망이다.']

In [ ]:
query='뉴진스 컴백 앨범 알려줘'
docs = db_search(query, 3)
docs = [doc.page_content for doc in docs]
docs

["하이브의 아티스트 뉴진스의 발매 앨범은 'Get Up'이고, 발매일은 0.0721이며, 미니 2집 (7/7, 선공개곡 발표)으로 구분됩니다. 하이브의 아티스트 뉴진스의 발매 앨범",
 "하이브의 '뉴진스'의 2023년 'Get Up'은 '0.0721'에 '미니 2집 (7/7, 선공개곡 발표)'로 발매되었으며, 판매량은",
 "구분됩니다. 하이브의 아티스트 뉴진스의 발매 앨범 'Get Up' 판매량은 1969969장입니다."]

In [ ]:
# 합쳐진 리스트를 세트로 변환하여 중복을 제거

docs_all=set(docs_bm25+docs)
len(docs_all)

5



---
프롬포트


---



In [ ]:
# prompt template
template = '''
    너는 사실만을 말하는 금융 전문가야.
    질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 대답해.
    문서에서 질문에 대한 근거를 찾을 수 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 답해.
    문서 = {docs_input}
    질문 = {query_input}
'''


In [ ]:
# prompt
prompt = PromptTemplate(
    input_variables=[
        "docs_input",
        "query_input",
    ],
    template=template,)



---

BM25


---



In [ ]:
# BM25 알고리즘을 사용하여 두 개의 다른 코퍼스에 대한 검색 객체를 생성
# 각각의 검색 결과에서 상위 4개의 문서를 반환

from langchain.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_texts(corpus)
bm25_retriever.k = 4
bm25_retrieverg = BM25Retriever.from_texts(corpus2)
bm25_retrieverg.k = 4

In [ ]:
bm25_retriever.get_relevant_documents("현대 자동차 주가 요약")

[Document(page_content='퀄컴의 52주 주가 최저는 101.47 달러였습니다.\n퀄컴의 52주 주가 최고는 139.94 달러였습니다.\u200b\n퀄컴의 90일 평균거래대금은 0.9 십업 달러입니다.'),
 Document(page_content='하이브의 주당 적정가치는 330,105원이다.\n하이브의 목표주가는 330,000원이다.\n하이브의 현재 주가 (11/2)는 227,500십억원이다.'),
 Document(page_content='테슬라는 자동차 섹터에 속해 있으며, S&P 지수에서 6.3%의 상승을 기록하여 상위 종목 중 하나가 되었습니다.'),
 Document(page_content='5,724대(+31.5%YoY) 판매로 호조다. 고금리 영향 누적으로 자동차 수요에 대한 불안감 증폭. 4분기 현대차/기아의 연중 최대 생산 및 최대 판매로 우려 불식이')]

In [ ]:
bm25_retriever.get_relevant_documents("퀄컴 주가 요약")

[Document(page_content='퀄컴의 52주 주가 최저는 101.47 달러였습니다.\n퀄컴의 52주 주가 최고는 139.94 달러였습니다.\u200b\n퀄컴의 90일 평균거래대금은 0.9 십업 달러입니다.'),
 Document(page_content='하이브의 주당 적정가치는 330,105원이다.\n하이브의 목표주가는 330,000원이다.\n하이브의 현재 주가 (11/2)는 227,500십억원이다.'),
 Document(page_content='두번째, 내년 차기 Galaxy S 시리즈는 퀄컴 AP를 전량 탑재할 전망이 아니어서 퀄컴의 Galaxy S 내 점유율 하락될것이기 때문이다.'),
 Document(page_content='퀄컴의 요약 손익 중 세전이익률은 2022년에 37.6% 입니다.\n퀄컴의 요약 손익 중 세전이익률은 2023년에 30.9% 입니다.')]



---

faiss


---



In [ ]:
faiss_retriever = dbf.as_retriever(search_kwargs={"k": 7})
faiss_retrieverg = dbg.as_retriever(search_kwargs={"k": 7})

In [ ]:
faiss_retriever.get_relevant_documents('롯데 칠성 목표 주가')

[Document(page_content='롯데칠성은 맥주 신제품 판매 성과에 따라, 추가적인 밸류에이션 눈높이 상향도 가능할 것으로 보입니다. 롯데칠성은 글로벌 확장성을 보유한 동사의 밸류에이션 눈높이가 현재 24E', metadata={'source': '/content/drive/MyDrive/finaltexts/롯데칠성_pdf 텍스트 전처리.txt'}),
 Document(page_content='롯데칠성은 글로벌 확장성을 보유한 동사의 밸류에이션 눈높이가 상향되어야 할 것으로 예상합니다. 한편, 롯데칠성은 11월 21일 출시 예정인 맥주 신제품이 추가적인 밸류에이션', metadata={'source': '/content/drive/MyDrive/finaltexts/롯데칠성_pdf 텍스트 전처리.txt'}),
 Document(page_content='롯데칠성은 업종 내 원재료 가격 상승에 대한 압박이 가장 높은 업체이나, 이를 뛰어넘는 점유율 확대를 시현 중인 것으로 판단합니다.', metadata={'source': '/content/drive/MyDrive/finaltexts/롯데칠성_pdf 텍스트 전처리.txt'}),
 Document(page_content="롯데칠성의 투자의견은 'BUY'입니다.\n롯데칠성의 목표주가는 190,000원입니다.\n롯데칠성의 현재가격은 149,300원입니다.\n롯데칠성의 시가총액은 1.4조원입니다.", metadata={'source': '/content/drive/MyDrive/finaltexts/롯데칠성_AT A GLANCE.txt'}),
 Document(page_content='종합적으로 보면, 롯데칠성은 매출, 영업이익, 순이익 등 주요 재무 지표들에서 지속적인 성장을 예상하고 있으며, 이는 재무 건전성과 수익성이 향상될 것임을 나타내는 긍정적인', metadata={'source': '/content/drive

In [ ]:
faiss_retriever.get_relevant_documents('퀄컴 주가 요약')

[Document(page_content='퀄컴은 반도체 & 반도체 장비 섹터의 중요한 기업으로, 이번 마감시황에서 수익률이 5.8%로 상승하였습니다.', metadata={'source': '/content/drive/MyDrive/finaltexts/미국 마감시황_S_P 수익률 상하위 종목_UP.txt'}),
 Document(page_content='퀄컴의 EPS (지배주주)는 2020년에 4.52달러 입니다.\n퀄컴의 EPS (지배주주)는 2021년에 7.87달러 입니다.', metadata={'source': '/content/drive/MyDrive/finaltexts/퀄컴_재무비율 및 주당지표.txt'}),
 Document(page_content='퀄컴의 순이익은 2018년에 -4,964백만달러 입니다.\n퀄컴의 순이익은 2019년에 4,386백만달러 입니다.\n퀄컴의 순이익은 2020년에 5,198백만달러 입니다.', metadata={'source': '/content/drive/MyDrive/finaltexts/퀄컴_포괄손익계산서.txt'}),
 Document(page_content='퀄컴은 세부적으로, Automotive가 가장 가파른 성장률(23% q-q)을 기록했고, 가장 비중이 큰 Handsets도 4% 성장했다.', metadata={'source': '/content/drive/MyDrive/finaltexts/퀄컴_pdf 텍스트.txt'}),
 Document(page_content='퀄컴의 실적과 가이던스 모두 컨센서스를 상회했습니다. 퀄컴은 마침내 업황 개선과 매출 성장 재개를 전망합니다.', metadata={'source': '/content/drive/MyDrive/finaltexts/퀄컴_pdf 텍스트.txt'}),
 Document(page_content='퀄컴의 지배주주지분은 2018년에 9



---

ensemble


---



In [ ]:
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, faiss_retriever],
                                       weights=[0.3, 0.7])

In [ ]:
docs = ensemble_retriever.get_relevant_documents("현대 자동차 목표주가 예상")
docs = [doc.page_content for doc in docs]
docs

['5,724대(+31.5%YoY) 판매로 호조다. 고금리 영향 누적으로 자동차 수요에 대한 불안감 증폭. 4분기 현대차/기아의 연중 최대 생산 및 최대 판매로 우려 불식이',
 '연중 최대 생산 및 최대 판매로 우려 불식이 예상된다. Value Chain에도 관심이 필요한 시기다. 현대차, 글로벌 도매 판매 37.8만 대(+5.8%MoM,',
 '현대차 미국 소매 판매, 66,529대(-11.8%MoM, +2.4%YoY) 및 M/S 5.5%은 주요 모델별로 팰리세이드 7,071대/ 싼타페 9,700대/ 투싼',
 '현대차 (005380 KS, 170,300)의 투자의견은 BUY이다. 현대차의 목표 주가는 260000원이고 52.7%이다.',
 '현대의 완성차 판매 총계 2023년 10월 377,986대이다.\n현대의 완성차 판매 총계 2022년 10월 344,910대이다.',
 '현대차 66,529대(+2.4%YoY), 기아 59,164대(+1.5%YoY). 아이오닉 5/6은 도합 4,218대, 제네시스는 5,724대(+31.5%YoY) 판매로 호조다.',
 '현대의 아이오닉_5의 2023년 10월 판매 실적은 1,471대이다.\n현대의 아이오닉_5의 2022년 10월 판매 실적은 2,169대이다.',
 '테슬라는 자동차 섹터에 속해 있으며, S&P 지수에서 6.3%의 상승을 기록하여 상위 종목 중 하나가 되었습니다.',
 '롯데칠성의 목표주가는 새로운 목표주가는 190,000원으로, 이전 목표주가 140,000원에 비해 35.7% 상승하였습니다.',
 '하이브의 새로운 목표주가가 330,000원입니다.\n하이브의 이전 목표주가는 360,000원입니다.\n하이브의 목표주가 변동률은 -8.3%입니다.']



---
LLM


---




In [ ]:
import torch

from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TextStreamer, GenerationConfig

def load_local_llm():
  model='davidkim205/komt-mistral-7b-v1'
  peft_model_name = 'davidkim205/komt-mistral-7b-v1-lora'
  config = PeftConfig.from_pretrained(peft_model_name)
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )
  config.base_model_name_or_path =model
  model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map="auto")
  model = PeftModel.from_pretrained(model, peft_model_name)
  return model

In [ ]:
model=load_local_llm()

adapter_config.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/84.0M [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(PeftConfig.from_pretrained('davidkim205/komt-mistral-7b-v1-lora').base_model_name_or_path)
streamer = TextStreamer(tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
def load_chain(_prompt) :
    chain = LLMChain(
        llm=model,
        prompt=prompt
    )
    return chain



---
query 실행


---




In [ ]:
query='뉴진스의 앨범 판매량 알려줘'
docs = db_search(query, SEARCH_NUM)
docs = [doc.page_content for doc in docs]
docs

["구분됩니다. 하이브의 아티스트 뉴진스의 발매 앨범 'Get Up' 판매량은 1969969장입니다.",
 "하이브의 아티스트 뉴진스의 발매 앨범은 'Get Up'이고, 발매일은 0.0721이며, 미니 2집 (7/7, 선공개곡 발표)으로 구분됩니다. 하이브의 아티스트 뉴진스의 발매 앨범",
 "하이브의 '뉴진스'의 2023년 'Get Up'은 '0.0721'에 '미니 2집 (7/7, 선공개곡 발표)'로 발매되었으며, 판매량은"]

In [ ]:
docs_bm25=bm25.get_top_n(make_tok(query),corpus,n=3)
docs_bm25

["구분됩니다. 하이브의 아티스트 뉴진스의 발매 앨범 'Get Up' 판매량은 1969969장입니다.",
 "하이브의 아티스트 뉴진스의 발매 앨범은 'Get Up'이고, 발매일은 0.0721이며, 미니 2집 (7/7, 선공개곡 발표)으로 구분됩니다. 하이브의 아티스트 뉴진스의 발매 앨범",
 'TXT, 세븐틴, 정국의 해외향 음반 판매량 일부(약 200억원)가 3분기에 반영됐다.']

In [ ]:
docs_all=set(docs+docs_bm25)
docs_all

{'TXT, 세븐틴, 정국의 해외향 음반 판매량 일부(약 200억원)가 3분기에 반영됐다.',
 "구분됩니다. 하이브의 아티스트 뉴진스의 발매 앨범 'Get Up' 판매량은 1969969장입니다.",
 "하이브의 '뉴진스'의 2023년 'Get Up'은 '0.0721'에 '미니 2집 (7/7, 선공개곡 발표)'로 발매되었으며, 판매량은",
 "하이브의 아티스트 뉴진스의 발매 앨범은 'Get Up'이고, 발매일은 0.0721이며, 미니 2집 (7/7, 선공개곡 발표)으로 구분됩니다. 하이브의 아티스트 뉴진스의 발매 앨범"}

In [ ]:
def findcore2(x):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=300,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        다음 질문과 가장 관련있는 명사를 모두 고르시오.
        질문: {x}
        반드시 아래에 있는 명사만을 선택하시오.
        답변에는 아래에 존재하는 명사 이외의 말은 절대로 하지 마시오.

        다음 5가지 명사 중에서 모두 고르시오. : '하이브', '미국', '자동차', '퀄컴', '롯데칠성'
        [/INST]

    """

    q = template

    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

In [ ]:
def findcore(x):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=300,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        질문의 핵심 단어 말해줘.
        반드시 질문에 있는 단어를 말해줘.

        질문: {x} [/INST]

    """

    q = template

    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

In [ ]:
def gen1(x,docs_input):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=200,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        너는 사실만을 말하는 금융 전문가야.
        질문 = {x} [/INST]
    """

    q = template
    # q = f"[INST]{x} [/INST]"
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

In [ ]:
def graphs(x,docs_input):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=300,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        아래 문서는 그래프에 대한 해석이다.
        문서에 주어진 내용만을 이용해서 답하고 문서에서 근거를 찾을 수 없거나 답변하기 모호하면 정보를 찾을 수 없습니다. 라고 답변해줘.

        문서: {docs_input}
        질문: {x} [/INST]
        output (주어진 질문과 유사한 질문 2개):
    """

    q = template

    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

In [ ]:
def makesimilarquery(x):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=300,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        질문과 관련된 2개의 다중 검색 질문를 생성해줘.

        질문: {x} [/INST]
        output (주어진 질문과 유사한 질문 2개):
    """

    q = template

    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

In [ ]:
def find_source(x):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=100,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        너는 주어진 질문에 답하기 위해서 필요한 문서를 찾아야해.
        주어진 5개의 예시 중에서 필요한 문서를 골라줘.
        예시: 하이브, 자동차, 퀄컴, 롯데칠성, 미국 마감 사항
        대답은 위의 5개의 예시중에서 고르고 짧게 명칭만 말해줘.
        예를 들어
        질문: 현대차와 기아차의 차이를 알려줘
        답변: 자동차

        질문: {x} [/INST]
    """

    q = template

    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

In [ ]:
def gen0(x,docs_input):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=300,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        너는 사실만을 말하는 금융 전문가야.
        질문이 주어진 문서의 내용과 관련이 있다면, 해당 문서를 바탕으로 구체적이고 명확한 답변을 해줘.
        모든 문서의 내용을 이해하고 고려해서 질문과 관련된 정보를 모두 찾아 답변 해줘.
        만약 질문에 '모두', '전부'와 같은 단어가 포함되어 있다면 해당하는 정보를 문서에서 모두 최대한 찾아서 답변해줘. 문서에 존재하지 않는 내용은 답하면 안돼.
        질문에 연도가 포함되어 있다면, 그 연도에 해당하는 정확한 데이터를 제공해줘.
        만약 질문에 연도가 명시되지 않았다면, 현재 연도인 2023년의 정보를 사용해줘.
        국내 시장과 관련된 질문이면 '내수'로 이해하고 답변해줘.
        비율이나 성장률, 이익률 등에 대한 질문에는 그에 해당하는 비율 값을 제공해줘.
        대답은 완성된 문장으로 해주고 완성하지 못할것 같으면 이전 문장까지만 만들어줘.
        문서 내용과 관련 없거나 애매한 질문이면 '내용을 찾을 수 없습니다.'라고 답변해줘.
        소속이나 존재를 묻는 질문은 문서에 정확하게 일치하는 내용이 없다면 없다고 답변해줘
        필요한 답변만 하고 불필요한 답변은 생성하면 안돼.
        문서들: {docs_input}
        질문: {x} [/INST]
    """

    q = template

    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

In [ ]:
def gengraph(x,docs_input):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=300,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        질문과 가장 관련 있는 것을 선택해서 대답해줘.
        주어진 문서에 있는 것중 선택해서 명확하게 단답으로 말해줘.
        반드시 밑에 존재하는 문서에 존재하는 것 중 선택해서 대답해줘.

        문서: {docs_input}
        질문: {x} [/INST]
        output (선택된 문서 제목):
    """

    q = template

    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

In [ ]:
def gen(x,docs_input,chat_history):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=300,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        너는 사실만을 말하는 금융 전문가야.
        질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 대답해.
        질문한 연도와 같은 시점의 값만 말해줘.
        문서가 주어질때 주어가 없다면 질문과 매우 관련이 높은 주어라고 생각하고 답해줘.
        대답을 한 근거를 생각하고 차근차근 판단해줘. 대답은 존댓말로 해줘.
        질문에 대해 대답할 때 사용자의 질문이 수치에 관련된 것이라면 비율 %에 대해 답하지 말고 정확한 값을 대답해줘.
        년도에 E가 붙어 있으면 기대값이야.
        비율에 관한 질문이면 성장률 이익률 등 비율에 관해서 대답해줘
        현재 질문에 대해 답변을 하고 현재 질문에 대해 충분한 대답이 어려울 때만 이전 대화를 고려해서 주어와 정보를 찾아 대답해줘.
        문서에 질문에 답변할 정보가 없다면 죄송합니다. 내용을 찾을 수 없습니다.라고 대답해.
        문서 = {docs_input}
        질문 = {x}
        이전 대화 = {chat_history} [/INST]

    """

    q = template

    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

In [ ]:
def genllama2(x,docs_input):
    generation_config = GenerationConfig(
        temperature=0.8,
        top_p=0.8,
        top_k=100,
        max_new_tokens=512,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""### instruction:
        너는 사실만을 말하는 금융 전문가야.
        질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 대답해.
        문서 = {docs_input}
        질문 = {x}\n\n### Response:
    """

    q = template

    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"\n\n### Response: "
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str


In [ ]:
def test1(x,docs_input):
    generation_config = GenerationConfig(
        temperature=0.8,
        top_p=0.8,
        top_k=100,
        max_new_tokens=100,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        주어진 문서 내에서 다음 질문에 답할 수 있는 정보가 포함되어 있는지 판단하라.
        주어진 문서에서 다음 질문에 답할 근거가 충분하지 않다면 'N'을 충분하다면 'Y'를 출력해라.
        문서 = {docs_input}
        질문 = {x} [/INST]
        output (Y or N):
    """

    q = template
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str


In [ ]:
def testlamma2(x,docs_input):
    generation_config = GenerationConfig(
        temperature=0.8,
        top_p=0.8,
        top_k=100,
        max_new_tokens=100,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""### instruction:
        주어진 문서 내에서 다음 질문에 답할 수 있는 정보가 포함되어 있는지 판단하라.
        문서 = {docs_input}
        질문 = {x} 주어진 문서에서 다음 질문에 답할 근거가 충분한가? \n\n### Response:

    """

    q = template
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"\n\n### Response: "
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str


In [ ]:
query='현대 자동차 내수시장성과 설명'
outputs=findcore2(query)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:422: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:422: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


<s> [INST]
        다음 질문과 가장 관련있는 명사를 모두 고르시오.
        질문: 현대 자동차 내수시장성과 설명
        반드시 아래에 있는 명사만을 선택하시오.
        답변에는 아래에 존재하는 명사 이외의 말은 절대로 하지 마시오.

        다음 5가지 명사 중에서 모두 고르시오. : '하이브', '미국', '자동차', '퀄컴', '롯데칠성'
        [/INST]

    설명과 관련된 명사: 설명
  현대 자동차 내수시장성과 설명: 내수시장</s>


In [ ]:
from langchain.document_transformers import (
    LongContextReorder,
)



---
query


---




In [ ]:
query='벨로스터는 어느 회사 차야?'
outputs=makesimilarquery(query)
core=findcore(query)
docs = ensemble_retriever.get_relevant_documents(query)
#docs = [doc.page_content for doc in docs]
docs4 =ensemble_retriever.get_relevant_documents(core)
#docs4 = [doc.page_content for doc in docs4]
docs2 =ensemble_retriever.get_relevant_documents(outputs.split('\n')[1])
#docs2 = [doc.page_content for doc in docs2]
docs3 =ensemble_retriever.get_relevant_documents(outputs.split('\n')[2].replace('</s>',''))
#docs3 = [doc.page_content for doc in docs3]

docs_all=(docs+docs2+docs3+docs4)
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs_all)
reordered_docs = [doc.page_content for doc in reordered_docs]
reordered_docs=set(reordered_docs)
gen0(query,reordered_docs)

<s> [INST]
        질문과 관련된 2개의 다중 검색 질문를 생성해줘.

        질문: 벨로스터는 어느 회사 차야? [/INST]
        output (주어진 질문과 유사한 질문 2개):
    
       - 벨로스터는 어느 회사에 소속되어 있나요?
       - 벨로스터는 어느 차 브랜드에 속하는 회사인가요?</s>
<s> [INST]
        질문의 핵심 단어 말해줘.
        반드시 질문에 있는 단어를 말해줘.

        질문: 벨로스터는 어느 회사 차야? [/INST]

    벨로스터</s>


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:422: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (2595 > 1024). Running this sequence through the model will result in indexing errors


<s> [INST]
        너는 사실만을 말하는 금융 전문가야.
        질문이 주어진 문서의 내용과 관련이 있다면, 해당 문서를 바탕으로 구체적이고 명확한 답변을 해줘.
        모든 문서의 내용을 이해하고 고려해서 질문과 관련된 정보를 모두 찾아 답변 해줘.
        만약 질문에 '모두', '전부'와 같은 단어가 포함되어 있다면 해당하는 정보를 문서에서 모두 최대한 찾아서 답변해줘. 문서에 존재하지 않는 내용은 답하면 안돼.
        질문에 연도가 포함되어 있다면, 그 연도에 해당하는 정확한 데이터를 제공해줘.
        만약 질문에 연도가 명시되지 않았다면, 현재 연도인 2023년의 정보를 사용해줘.
        국내 시장과 관련된 질문이면 '내수'로 이해하고 답변해줘.
        비율이나 성장률, 이익률 등에 대한 질문에는 그에 해당하는 비율 값을 제공해줘.
        대답은 완성된 문장으로 해주고 완성하지 못할것 같으면 이전 문장까지만 만들어줘.
        문서 내용과 관련 없거나 애매한 질문이면 '내용을 찾을 수 없습니다.'라고 답변해줘.
        소속이나 존재를 묻는 질문은 문서에 정확하게 일치하는 내용이 없다면 없다고 답변해줘
        필요한 답변만 하고 불필요한 답변은 생성하면 안돼.
        문서들: {"하이브의 '르세라핌'의 2023년 '2023 LE SSERAFIM TOUR ‘FLAME RISES’'은 '아시아'에서 '오프라인'으로 '10.1~10.8'에 4회", 'Porsche의 통화는 EUR이고, Porsche의 주가는 83.0입니다. Porsche의 1개월 성과는 -6.8%, 3개월 성과는 -26.0%, 1년 성과는 -20.8%입니다.', "하이브의 '황민현'의 2023년 'Hwang Min Hyun Mini Concert 'Unveil''은 '아시아'에서 '오프라인'으로 '8.4~9.2'에 7회", "하이브의 '황민현'의 2023년 'Hwang Min Hyun Mini C

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:422: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


현대의 벨로스터</s>


'현대의 벨로스터</s>'

In [ ]:
reordered_docs

{'+8.3%YoY) 판매되며, 니로(1,813대/ +1.6%MoM, +10.4%YoY), 쏘렌토(8,777대/ -13.9%MoM, +71.2%YoY)와 EV6(1,564대/',
 '2023년에 nan배, 2024년에 nan배, 2025년에 nan배입니다. Kraneshares_Electric_Vehicle의 ROE는 2023년에 nan%, 2024년에',
 'Porsche의 통화는 EUR이고, Porsche의 주가는 83.0입니다. Porsche의 1개월 성과는 -6.8%, 3개월 성과는 -26.0%, 1년 성과는 -20.8%입니다.',
 'ROE는 2023년에 nan%, 2024년에 nan%, 2025년에 nan%입니다.',
 '기아의 RV의 10월 판매 실적은 전년비(%) 8.3이다.\n기아의 RV의 2023년 9월 판매 실적은 28222대이다.',
 '성과는 -21.2%, 1년 성과는 -9.7%입니다. Tesla의 시가총액은 현지 통화로 653.8억, USD로는 653.8억입니다. Tesla의 2023년 예상 PER는',
 '성과는 -26.0%, 1년 성과는 -20.8%입니다. Porsche의 시가총액은 현지 통화로 75.6억, USD로는 79.9억입니다. Porsche의 2023년 예상 PER는',
 '성과는 -34.4%, 1년 성과는 -23.3%입니다. Nexteer의 시가총액은 현지 통화로 9.2억, USD로는 1.2억입니다. Nexteer의 2023년 예상 PER는',
 '주가는 23.7입니다. Kraneshares_Electric_Vehicle의 1개월 성과는 -13.8%, 3개월 성과는 -29.3%, 1년 성과는 -22.9%입니다.',
 '퀄컴의 영업이익률은 2018년에 2.7% 입니다.\n퀄컴의 영업이익률은 2019년에 31.6% 입니다.\n퀄컴의 영업이익률은 2020년에 26.6% 입니다.',
 '퀄컴의 영업이익은 2021년에 9,789백만달러 입니다.\n퀄컴의 영업이익은 2022년에 15,860백만달러 입니다.',
 '퀄컴의 판매 및 일

In [ ]:
query='하이브 아시아 공연 알려줘'

docs = db_search(query, 10)
docs = [doc.page_content for doc in docs]
docs2 =db_search2(query, 10)
docs2 = [doc.page_content for doc in docs2]
docs_bm25=bm25.get_top_n(make_tok(query),corpus,n=3)
docs_all=set(docs+docs2+docs_bm25)
gen0(query,docs_all)

<s> [INST]
        너는 사실만을 말하는 금융 전문가야.
        질문이 주어진 문서의 내용과 관련이 있다면, 해당 문서를 바탕으로 구체적이고 명확한 답변을 해줘.
        모든 문서의 내용을 이해하고 고려해서 질문과 관련된 정보를 모두 찾아 답변 해줘.
        만약 질문에 '모두', '전부'와 같은 단어가 포함되어 있다면 해당하는 정보를 문서에서 모두 최대한 찾아서 답변해줘. 문서에 존재하지 않는 내용은 답하면 안돼.
        질문에 연도가 포함되어 있다면, 그 연도에 해당하는 정확한 데이터를 제공해줘.
        만약 질문에 연도가 명시되지 않았다면, 현재 연도인 2023년의 정보를 사용해줘.
        국내 시장과 관련된 질문이면 '내수'로 이해하고 답변해줘.
        비율이나 성장률, 이익률 등에 대한 질문에는 그에 해당하는 비율 값을 제공해줘.
        대답은 완성된 문장으로 해주고 완성하지 못할것 같으면 이전 문장까지만 만들어줘.
        문서 내용과 관련 없거나 애매한 질문이면 '내용을 찾을 수 없습니다.'라고 답변해줘.
        소속이나 존재를 묻는 질문은 문서에 정확하게 일치하는 내용이 없다면 없다고 답변해줘
        필요한 답변만 하고 불필요한 답변은 생성하면 안돼.
        문서들: {'하이브의 위버스 ARPPU는 2022년에 들어서면서 상승하는 모습을 보입니다.', '이 막대 그래프는 하이브의 위버스 플랫폼에 입점한 커뮤니티의 수를 2020년 1분기부터 2023년 3분기까지 나타냅니다.', '또한, 하이브의 위버스 플랫폼의 사용자 참여도를 높이고 다양한 콘텐츠와 상호작용을 제공하는데 중점을 두고 있음을 반영합니다.', '이 그래프는 하이브(HYBE)의 지난 2년간 목표 주가 변경 추이를 나타냅니다. \n파란색 선은 실제 주가를, 분홍색 선은 시간에 따른 목표 주가를 나타냅니다.', '하이브 위버스 MAU는 2022년에 들어서면서 성장세는 가속화되어 2022년 2분

"하이브의 아시아 공연은 2023년 3분기에 열리는 '르세라핌'의 '2023 LE SSERAFIM TOUR 'FLAME RISES'이며, 총 9회의 공연이 오프라인으로 개최됩니다. 이러한 성장은 하이브가 제공하는 콘텐츠의 인기, 플랫폼의 글로벌 확장 전략, 그리고 타깃 오디언스와의 강력한 커뮤니케이션에 기인할 수 있습니다.</s>"

In [ ]:
docs_all

{'또한, 하이브의 위버스 플랫폼의 사용자 참여도를 높이고 다양한 콘텐츠와 상호작용을 제공하는데 중점을 두고 있음을 반영합니다.',
 '이 그래프는 하이브(HYBE)의 소셜 미디어 플랫폼인 위버스(Weverse)의 월간 활성 사용자 수(Monthly Active Users, MAU)의 추이를 나타냅니다.',
 '이 그래프는 하이브(HYBE)의 지난 2년간 목표 주가 변경 추이를 나타냅니다. \n파란색 선은 실제 주가를, 분홍색 선은 시간에 따른 목표 주가를 나타냅니다.',
 '이 막대 그래프는 하이브의 위버스 플랫폼에 입점한 커뮤니티의 수를 2020년 1분기부터 2023년 3분기까지 나타냅니다.',
 '이 추세는 하이브의 위버스 플랫폼이 사용자 기반을 확장하고 있으며, 특히 2021년과 2022년 사이에 사용자 증가율이 더욱 두드러지는 것을 보여줍니다.',
 '이러한 성장은 하이브가 제공하는 콘텐츠의 인기, 플랫폼의 글로벌 확장 전략, 그리고 타깃 오디언스와의 강력한 커뮤니케이션에 기인할 수 있습니다.',
 '하이브 공연 매출액은 1Q23에는 25.2십억원입니다.\n하이브 공연 매출액은 2Q23에는 157.5십억원입니다.\n하이브 공연 매출액은 3Q23에는 86.9십억원입니다.',
 '하이브 공연 매출액은 2Q22에는 85십억원입니다.\n하이브 공연 매출액은 3Q22에는 47.2십억원입니다.\n하이브 공연 매출액은 4Q22에는 64.7십억원입니다.',
 '하이브 공연 매출액은 4Q23E에는 92.2십억원입니다.\n하이브 공연 매출액은 2021년에는 45.3십억원입니다.\n하이브 공연 매출액은 2022년에는 258.2십억원입니다.',
 '하이브 위버스 MAU는 2022년에 들어서면서 성장세는 가속화되어 2022년 2분기에는 6.9백만 명, 3분기에는 8.4백만 명으로 증가합니다.',
 "하이브의 '르세라핌'의 2023년 '2023 LE SSERAFIM TOUR ‘FLAME RISES’'은 '아시아'에서 '오프라인'으로 '10.1~10.8'에 4회",
 "하이브의

In [ ]:
import os

path = '/content/drive/MyDrive/finalimages/'

file_list = os.listdir(path)
file_list2=[]
for i in file_list:
  j=i.replace('_',' ')
  j=j.replace('.png','')
  file_list2.append(j)
file_list2

['하이브 그림 TXT 정규 3집 (10 13).PNG',
 '하이브 그림 더 데뷔 드림아카데미 방영 시작 (9 1 11 18).PNG',
 '하이브 그림 위버스 바이 팬즈(by Fans) 정식 론칭(10 16).PNG',
 '하이브 그림 빌리프랩 6인조 신인 걸그룹 I’LL-IT (아일릿).PNG',
 '하이브 그림 내년 출시 예정인 위버스 구독 서비스.PNG',
 '하이브 그림 정국 솔로 앨범 (11 3).PNG',
 '하이브 그림 세븐틴 미니 11집 (10 23).PNG']

In [ ]:
query='하이브 커뮤니티수 알려줘'
gengraph(query,file_list2)

<s> [INST]
        질문과 가장 관련 있는 것을 선택해서 대답해줘.
        주어진 문서에 있는 것중 선택해서 명확하게 단답으로 말해줘.
        반드시 밑에 존재하는 문서에 존재하는 것 중 선택해서 대답해줘.

        문서: ['하이브 그림 TXT 정규 3집 (10 13).PNG', '하이브 그림 더 데뷔 드림아카데미 방영 시작 (9 1 11 18).PNG', '하이브 그림 위버스 바이 팬즈(by Fans) 정식 론칭(10 16).PNG', '하이브 그림 빌리프랩 6인조 신인 걸그룹 I’LL-IT (아일릿).PNG', '하이브 그림 내년 출시 예정인 위버스 구독 서비스.PNG', '하이브 그림 정국 솔로 앨범 (11 3).PNG', '하이브 그림 세븐틴 미니 11집 (10 23).PNG']
        질문: 하이브 커뮤니티수 알려줘 [/INST]
        output (선택된 문서 제목):
    하이브 그림 TXT 정규 3집 (10 13).PNG</s>


'output (선택된 문서 제목):\n    하이브 그림 TXT 정규 3집 (10 13).PNG</s>'

In [ ]:
def db_searchs(query: str, k: int):
    docs = dbfs.similarity_search(query, k)
    return docs

In [ ]:
query='기아차와 현대차의 비교를 해줘'

source0=find_source(query)
source1=source0.split(':')[-1].split('<')[0]
source1

<s> [INST]
        너는 주어진 질문에 답하기 위해서 필요한 문서를 찾아야해.
        주어진 5개의 예시 중에서 필요한 문서를 골라줘.
        예시: 하이브, 자동차, 퀄컴, 롯데칠성, 미국 마감 사항
        대답은 위의 5개의 예시중에서 고르고 짧게 명칭만 말해줘.
        예를 들어
        질문: 현대차와 기아차의 차이를 알려줘
        답변: 자동차

        질문: 기아차와 현대차의 비교를 해줘 [/INST]
    대답은 위의 5개의 예시중에서 고르고 짧게 명칭만 말해줘야 합니다.

   질문: 현대차와 기아차의 차이를 알려줘
   답변: 자동차

   질문: 기아차와 현대차의 비교를 해


' 기아차와 현대차의 비교를 해'

In [ ]:
query='하이브의 법인세는?'

docs = db_search(query, 20)
docs = [doc.page_content for doc in docs]
docs_bm25=bm25.get_top_n(make_tok(query),corpus,n=10)
docs_all=set(docs)

gen0(query,docs_all)

<s> [INST]
        너는 사실만을 말하는 금융 전문가야.
        질문이 주어진 문서의 내용과 관련이 있다면, 해당 문서를 바탕으로 구체적이고 명확한 답변을 해줘.
        모든 문서의 내용을 이해하고 고려해서 질문과 관련된 정보를 모두 찾아 답변 해줘.
        만약 질문에 '모두', '전부'와 같은 단어가 포함되어 있다면 해당하는 정보를 문서에서 모두 최대한 찾아서 답변해줘. 문서에 존재하지 않는 내용은 답하면 안돼.
        질문에 연도가 포함되어 있다면, 그 연도에 해당하는 정확한 데이터를 제공해줘.
        만약 질문에 연도가 명시되지 않았다면, 현재 연도인 2023년의 정보를 사용해줘.
        국내 시장과 관련된 질문이면 '내수'로 이해하고 답변해줘.
        비율이나 성장률, 이익률 등에 대한 질문에는 그에 해당하는 비율 값을 제공해줘.
        대답은 완성된 문장으로 해주고 완성하지 못할것 같으면 이전 문장까지만 만들어줘.
        문서 내용과 관련 없거나 애매한 질문이면 '내용을 찾을 수 없습니다.'라고 답변해줘.
        소속이나 존재를 묻는 질문은 문서에 정확하게 일치하는 내용이 없다면 없다고 답변해줘
        필요한 답변만 하고 불필요한 답변은 생성하면 안돼.
        문서들: {'하이브의 2022년 Div yield는 0.0%입니다.\n하이브의 2023E년 Div yield는 0.0%입니다.\n하이브의 2024E년 Div yield는 0.8%입니다.', '하이브의 배당성향 비율은 2021년도에 0.0%로 기록되었습니다. \n하이브의 배당성향 비율은 2022년도에 0.0%로 기록되었습니다.', '하이브의 지배주주지분 중 이익잉여금은 2025E년도에 1,026십억원입니다.', '하이브 지배주주순이익 YoY는 1Q21에는 22.0% 입니다.\n하이브 지배주주순이익 YoY는 2Q21에는 14.6% 입니다.', '퀄컴의 요약 손익 중 법인세율은 2022년 1분기에 1

'2023년 법인세율은 27.6%입니다.</s>'

In [ ]:
query='하이브의 주가 변동을 알려줘'

docsss = db_search(query, 10)
docs = [doc.page_content for doc in docsss]
#source=[doc.metadata['source'].split('/')[-1].split('.txt의 사본')[0] for doc in docsss]
source=[doc.metadata['source'].split('/')[-1].split('_')[0] for doc in docsss]
docss=[x+':'+y for x,y in zip(source,docs)]
docs_bm25=bm25.get_top_n(make_tok(query),corpus,n=3)
docs_all=set(docss+docs_bm25)
gen0(query,docs_all)

<s> [INST]
        너는 사실만을 말하는 금융 전문가야.
        질문이 주어진 문서의 내용과 관련이 있다면, 해당 문서를 바탕으로 구체적이고 명확한 답변을 해줘.
        모든 문서의 내용을 이해하고 고려해서 질문과 관련된 정보를 모두 찾아 답변 해줘.
        만약 질문에 '모두', '전부'와 같은 단어가 포함되어 있다면 해당하는 정보를 문서에서 모두 최대한 찾아서 답변해줘. 문서에 존재하지 않는 내용은 답하면 안돼.
        질문에 연도가 포함되어 있다면, 그 연도에 해당하는 정확한 데이터를 제공해줘.
        만약 질문에 연도가 명시되지 않았다면, 현재 연도인 2023년의 정보를 사용해줘.
        국내 시장과 관련된 질문이면 '내수'로 이해하고 답변해줘.
        비율이나 성장률, 이익률 등에 대한 질문에는 그에 해당하는 비율 값을 제공해줘.
        대답은 완성된 문장으로 해주고 완성하지 못할것 같으면 이전 문장까지만 만들어줘.
        문서 내용과 관련 없거나 애매한 질문이면 '내용을 찾을 수 없습니다.'라고 답변해줘.
        소속이나 존재를 묻는 질문은 문서에 정확하게 일치하는 내용이 없다면 없다고 답변해줘
        필요한 답변만 하고 불필요한 답변은 생성하면 안돼.
        문서들: {'하이브:하이브의 시가총액은 9.5조원이다.\n하이브의 Shares (float)는 41,652,097주 (38.7%)이다.\n하이브의 52주 최저 주가는 122,500원이다.', '하이브:하이브의 투자의견 및 목표주가 변경은 2021년 11월 5일 평균 괴리율은 -30.13%이며, 괴리율의 최대 또는 최소 값은 -5.91%이다.', '하이브:하이브의 투자의견 및 목표주가 변경은 2022년 6월 16일 평균 괴리율은 -40.73%이며, 괴리율의 최대 또는 최소 값은 -30.37%이다.', '하이브의 주당 적정가치는 330,105원이다.\n하이브의 목표주가는 330,00

'2023년 하이브의 52주 주가 변동을 알려드릴게요. 현재 정보를 바탕으로 하이브의 52주 주가 변동률을 찾아보겠습니다. 정확한 데이터를 확인하지 못했으나, 하이브의 52주 주가 변동률은 2023년 현재 주가와 전년도 52주 주가 사이의 비율로 계산할 수 있습니다. 하이브의 52주 주가 변동률은 현재 주가 대비 전년도 52주 주가 대비 비율로 계산됩니다. 이를 통해 2023년 하이브의 52주 주가 변동률을 알 수 있습니다. 하이브의 주가 변동률은 상대적으로 상승 또는 하락하는 정도를 나타냅니다'

In [ ]:
docsss

[Document(page_content='하이브의 시가총액은 9.5조원이다.\n하이브의 Shares (float)는 41,652,097주 (38.7%)이다.\n하이브의 52주 최저 주가는 122,500원이다.', metadata={'source': '/content/drive/MyDrive/finaltexts/하이브_AT A GLANCE.txt'}),
 Document(page_content='하이브의 적정 가치는 13,723.5십억원이다. \n하이브의 주식 수는 41,573천주이다.\n하이브의 주당 적정가치는 330,105원이다.', metadata={'source': '/content/drive/MyDrive/finaltexts/하이브_SOTP 밸류에이션.txt'}),
 Document(page_content='하이브의 투자의견 및 목표주가 변경은 2022년 6월 16일 평균 괴리율은 -40.73%이며, 괴리율의 최대 또는 최소 값은 -30.37%이다.', metadata={'source': '/content/drive/MyDrive/finaltexts/하이브_최근 투자의견 및 목표주가 변경 (수정주가 기준).txt'}),
 Document(page_content='하이브의 EPS (지배주주) 주당지표는 2025년도에는 6,974로 예상됩니다.', metadata={'source': '/content/drive/MyDrive/finaltexts/하이브_재무비율 및 주당지표.txt'}),
 Document(page_content='하이브 지배주주순이익은 3Q22에는 89.6십억원이다.\n하이브 지배주주순이익은 4Q22에는 -167.5십억원이다.\n하이브 지배주주순이익은 1Q23에는 24.3십억원이다.', metadata={'source': '/content/drive/MyDrive/finaltexts/하이브_실적 추이와 전망.tx

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install --upgrade langchain

In [ ]:
from rank_bm25 import BM25Okapi
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# 키워드 목록 정의
keywords = ['자동차', '하이브', '미국', '퀄컴', '롯데칠성']

# 디렉토리 경로 설정
directory_path = '/content/drive/MyDrive/finaltexts'

# 해당 디렉토리 내의 모든 파일 목록을 가져오기 및 데이터 로드
file_list = os.listdir(directory_path)
file_to_data = {}
for file_name in file_list:
    with open(os.path.join(directory_path, file_name), 'r', encoding='utf-8') as file:
        file_to_data[file_name] = file.read()

# 사용자의 질문에서 키워드 추출
def extract_keywords_from_query(query):
    extracted_keywords = [keyword for keyword in keywords if keyword in query]
    return extracted_keywords

# 불용어 제거 및 결과 정리
def clean_and_format_results(results):
    cleaned_results = []
    for result in results:
        result = result.replace('\n', ' ').replace('\t', ' ').replace('{}', '')
        sentences = result.split('. ')
        for sentence in sentences:
            sentence = ' '.join(sentence.split())
            if sentence:  # 빈 문장 제거
                cleaned_results.append(sentence)
    return '\n'.join(cleaned_results)

# 검색 함수 정의
def search(query):
    query_keywords = extract_keywords_from_query(query)
    relevant_data = [content for content in file_to_data.values() if any(keyword in content for keyword in query_keywords)]

    # 관련 데이터에 대한 새로운 BM25 인덱스 생성
    tokenized_corpus = [doc.split(" ") for doc in relevant_data]
    relevant_bm25 = BM25Okapi(tokenized_corpus)

    # BM25 검색
    bm25_docs = relevant_bm25.get_top_n(query.split(" "), relevant_data, n=5)

    # FAISS 검색 (선택된 파일에 대해서만)
    relevant_dbf = FAISS.from_texts(relevant_data, embeddings)
    faiss_docs = relevant_dbf.similarity_search(query, 5)
    faiss_docs_texts = [doc.page_content for doc in faiss_docs]

    # 결과 결합 및 불용어 제거
    combined_docs = set(bm25_docs + faiss_docs_texts)
    return clean_and_format_results(combined_docs)

# 사용자 질문 입력 및 검색 실행
query = input("질문을 입력하세요: ")
search_results = search(query)
print("검색 결과:\n", search_results)

질문을 입력하세요: 하이브 2023년 매출액 알려줘
검색 결과:
 검색 결과가 없습니다.
